In [46]:
import requests
import json
from bs4 import BeautifulSoup as bs
import time
import random
import re
import pandas as pd
import psycopg2
import json

In [47]:
conn = psycopg2.connect(dbname='pgstage', user='linguist', password='eDQGK0GCStlYlHNV', host='192.168.122.183')
cursor = conn.cursor()


In [127]:
def get_def_info_from_meta(word, from_lang, to_lang, print_ouput = False):
    #multisence_dict[word] = {'word_id': word_id, 'lemma_branch':[], 'wordforms_branch':{}}
    definitions = []
    same_page = False
    for index in range(1,3):
        url = "https://dictionary.cambridge.org/dictionary/" + from_lang + "-" + to_lang + "/" + word + "_" + str(index)
        #url = "https://dictionary.cambridge.org/dictionary/" + from_lang + "-" + to_lang + "/" + word
        if print_ouput:print(url)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8' 
        soup = bs(response.text, 'html.parser')
        current_definition = {}
        #print(soup.prettify())
        sensewords_list = []
        for link in soup.find_all("meta"):
            if "translate" in link['content']:
                try:
                    for w in link['content'].split(".")[0].split(":")[1].split(","):
                        sensewords_list.append(w.strip())
                except:
                    pass
                break
                    
                #print(link['content'].split("."))
        time.sleep(random.uniform(0.01,0.1))
    return set(sensewords_list)
get_def_info_from_meta("word", "english","french",print_ouput = True)


https://dictionary.cambridge.org/dictionary/english-french/word_1


ConnectionError: HTTPSConnectionPool(host='dictionary.cambridge.org', port=443): Max retries exceeded with url: /dictionary/english-french/word_1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11b4ef5d0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [124]:
def get_def_info(word, from_lang, to_lang, print_ouput = False):
    #multisence_dict[word] = {'word_id': word_id, 'lemma_branch':[], 'wordforms_branch':{}}
    definitions = []
    same_page = False
    for index in range(1,3):
        url = "https://dictionary.cambridge.org/dictionary/" + from_lang + "-" + to_lang + "/" + word + "_" + str(index)
        #url = "https://dictionary.cambridge.org/dictionary/" + from_lang + "-" + to_lang + "/" + word
        if print_ouput:print(url)
        headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8' 
        soup = bs(response.text, 'html.parser')
        current_definition = {}
        #print(soup.prettify())
        sensewords_list = []
        for link in soup.find_all("div", attrs = {'class':"def-body"}):
            #print(link.prettify())
            for l in link.find_all("span", attrs = {'class':"trans","lang":"fr"}):
                #print(l.text.strip(), len(l.text.strip().split()))
                if len(l.text.strip().split()) >= 1 and len(l.text.strip().split()) <= 2 and "!" not in l.text.strip().split() and "." not in l.text.strip().split() and "?" not in l.text.strip().split():
                    sensewords_list.append(l.text.strip())
            #print("=====")
        time.sleep(random.uniform(0.01,0.1))
        """
        for link in soup.find_all(["span","b"]):
    
            classes = link.get('class')
            if classes and "def" in classes:
                if link.text.strip() in definitions:
                    same_page = True
                    break
                if print_ouput: print("DEFINITION", link.text.strip())
                current_definition['definition'] = link.text.strip()
                definitions.append(link.text.strip())
                
            if classes and "def-body" in classes:
                if print_ouput:print("RUS_DEFIN", link.find("span", attrs = {'class':"trans", "lang":"fr"}).text.strip())
                current_definition['local_word'] = link.find("span", attrs = {'class':"trans", "lang":"fr"}).text.strip()
                current_definition['examples'] = []
                examples_count = 0
                for ex in link.find_all("span", attrs = {'class':"eg"}):
                    if examples_count > 3: break
                    current_definition['examples'].append(ex.text.strip())
                    examples_count += 1
                    if print_ouput:print("ENG_DEFIN_EX", ex.text.strip())
                if print_ouput:print(current_definition)
                multisence_dict[word]['lemma_branch'].append(current_definition)
                current_definition = {}
                if print_ouput:print("===")

        if same_page == True:
            break
        time.sleep(random.uniform(0.01,0.1))
        """
    if print_ouput: print(set(sensewords_list))
    return set(sensewords_list)
sence_dict = {}      
get_def_info("word", "english","french",print_ouput = False)
#sence_dict

ConnectionError: HTTPSConnectionPool(host='dictionary.cambridge.org', port=443): Max retries exceeded with url: /dictionary/english-french/word_1 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11c750bd0>: Failed to establish a new connection: [Errno 60] Operation timed out'))

In [45]:
words = pd.read_csv("test_words.csv", header = None)
words.head()
wds = []
for word in words[0].items():
    #print(word)
    wds.append(word[1])

In [48]:
def get_word_forms(word):
    req = """SELECT DISTINCT
    content_words.word_id,content_words.word_lemma, content_words.word_value
    FROM content_words
    WHERE content_words.word_value !~ ('\W') AND
    (array_length(regexp_split_to_array(content_words.word_value, '[ ''-]'), 1) = 1) 
    and word_lemma != 0 and word_hash = calc_hash(' """ + word + """')"""
    cursor.execute(req)
    req_res = cursor.fetchone()
    if not req_res:
        print(word, " is not in db")
        return []
    else:
        #print(a[0])
        word_id = req_res[0]
    word_forms_request = """SELECT DISTINCT 
    content_words.word_value
    FROM content_words
    where word_lemma = """ + str(word_id) + """  and word_lemma != word_id """
    cursor.execute(word_forms_request)
    word_forms = []
    for word in cursor:
        #print(word[0])
        word_forms.append(word[0])
    return word_forms
get_word_forms("book")

['booker', 'booked', 'booking', 'books']

In [123]:
sensewords_dict = {}
count = 0 
for word in words[0].items():
    print (word[1])
    sensewords_dict[word[1]] = get_def_info(word[1], "english","french",print_ouput = False)
    time.sleep(random.uniform(0.01,0.13))
    count += 1

word


KeyboardInterrupt: 

In [115]:
for key in sensewords_dict.keys():
    sensewords_dict[key] = list(sensewords_dict[key])
with open("oxford_sensewords.json", "w") as f:
    json.dump(sensewords_dict, f, indent = 4)

In [116]:
sensewords_dict_meta = {}
count = 0 
for word in words[0].items():
    print (word[1])
    sensewords_dict_meta[word[1]] = get_def_info_from_meta(word[1], "english","french",print_ouput = False)
    time.sleep(random.uniform(0.01,0.1))
    count += 1

word
the
and
that
for
with
but
you
this
not
have
letter
plate
treat
dinner
necessary
go
secret
table
create
warm
America
nobody
entire
thousand
Belgium
anything
background
child
childhood
dependable
everything
grandchild
ketchup
point
strength
star
thing
coincide
coincidental
dance
countryside
break
mother
start
tip
apologise
come
curriculum
expertise
freeze
issue
peremptory
parent
member
band
order
hot
fly
lie
select
school
cafe
buy
tell
international
stand
fish
benefit
intelligent
follow
journey
news
write
approbation
read
hear
feel
be
right
guru
beginner
collide
genetically
precedent
refugee
secular
pasta
beaver
tiger
lion
pen
coconut
dynamite
spoon
ocean
mug
shampoo
buzzer
exhaustively
apoplexy
submissively
typhus
ravioli
meeting
actually
earring
add
adjective
drawing
dream
dressed
driving
drum
dry
duck
advanced
during
earn
singular
easily
east
eighth
adventure
eighty
adverb
electric
advertisement
century
size
electricity
elephant
else
empty
engine
engineer
enough
enter
entrance
ad

In [117]:
for key in sensewords_dict_meta.keys():
    sensewords_dict_meta[key] = list(sensewords_dict[key])
with open("oxford_sensewords_meta.json", "w") as f:
    json.dump(sensewords_dict_meta, f, indent = 4)

In [107]:
sensewords_dict['word']

['mot', 'bruit [masculine] qui court', 'parole', 'formuler', 'nouvelles']